In [81]:
import re

import pandas as pd
import nltk
#window will open -> models -> punkt
nltk.download('nltk')

[nltk_data] Error loading nltk: Package 'nltk' not found in index


False

In [82]:
#read in the comments csv that we got from the reddit API
comments = pd.read_csv('comments_.csv')

In [83]:
sample = comments.Reply.iloc[35]
sample

'Thanks for everything.'

In [84]:
re.sub(r'\(?http\S+', '', sample)
sample

'Thanks for everything.'

In [85]:
#remove escape characters
s = "Salut \n Comment ca va ?"
s = re.sub(r'[\n\r\t]', '', s)
s

'Salut  Comment ca va ?'

In [86]:
#remove unicode
uni = 'ï ï ï ïqwkbbqheqbï'
uni = re.sub(r'[^\x00-\x7F]+',' ', uni)
uni

'       qwkbbqheqb '

In [87]:
#remove trailing whietspace after whitespace characters
text = '       qwkbbqheqb '
text = re.sub(r'^[ \t]+|[ \t]+$','', text)
text

'qwkbbqheqb'

In [88]:
great = comments[comments.Reply.str.contains("\n")]
great

,Reply,Upvote,Time,Key
5,MY FUCKING LEGEND\n\nALWAYS A BLUE,63.0,2019-05-30 08:59:32,1.0
12,We moved from the old guard era to the Eden ha...,12.0,2019-05-30 09:32:57,1.0
19,One of the most skilful and talented players w...,7.0,2019-05-30 09:28:08,1.0
20,"First trophy in Chelsea was EL, last trophy al...",6.0,2019-05-30 09:24:43,1.0
29,"As a belgian I'm so proud of this guy, he does...",5.0,2019-05-30 12:16:52,1.0
...,...,...,...,...
48691,>You make a good point about the difference be...,1.0,2020-05-30 10:26:22,499.0
48693,Those are official accounts with social media ...,0.0,2020-05-30 05:59:14,499.0
48695,You okay bro? You just replied to like 6 of my...,0.0,2020-05-30 06:15:48,499.0
48696,"Xenophobic, whatever you want to call it.\n\nI...",2.0,2020-05-30 22:59:09,499.0


In [89]:
comments.Reply = comments.Reply.apply(lambda x: re.sub(r'[\n\r\t]', '', x) )

In [90]:
#check that we have successfully removed escape characters
great = comments[comments.Reply.str.contains("\n")]
great

,Reply,Upvote,Time,Key


In [91]:
word_tokenized = nltk.word_tokenize(sample)
word_tokenized

['Thanks', 'for', 'everything', '.']

Bigrams

In [92]:
sample = ['more', 'is', 'said', 'than', 'done', 'is', 'said', 'than']

In [93]:
from nltk.collocations import *

bigram_measures = nltk.collocations.BigramAssocMeasures()

finder = BigramCollocationFinder.from_words(sample)

# only bigrams that appear 2+ times
finder.apply_freq_filter(2) 

# return the 10 n-grams with the highest PMI
print (finder.nbest(bigram_measures.pmi, 10))

[('is', 'said'), ('said', 'than')]


In [94]:
#connect all the first 10000 comments in the reply column into a single line separated by a period
all_text = comments.iloc[:10000,:].Reply.str.cat(sep='. ')

In [95]:
all_text = nltk.word_tokenize(all_text)

In [96]:
def common_bigrams(tokenized_text, min_freq, top_n):
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(tokenized_text)
    finder.apply_freq_filter(min_freq) 
    finder.nbest(bigram_measures.pmi, top_n)
    return finder.nbest(bigram_measures.pmi, top_n)

print(common_bigrams(all_text, 10, 10)) # top 10 bigrams

KeyboardInterrupt: 

Wordcloud

In [ ]:
#install wordcloud into the current path
import sys
!{sys.executable} -m pip install wordcloud

In [ ]:
import wordcloud
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

sample = comments.iloc[:10000,:].Reply.str.cat(sep='. ')

In [ ]:
import random
def grey_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(0, 0%%, %d%%)" % random.randint(1, 20)

In [ ]:
stp = STOPWORDS.copy()
#stp.add() #add stopwords to remove from the plot
wc = WordCloud(background_color="white", max_words=50,  stopwords=stp)
# generate word cloud
wc.generate(sample)

plt.figure(figsize=(10,20))
plt.imshow(wc.recolor(color_func=grey_color_func, random_state=3))
plt.show()

Vader Sentitment Analysis is good for analysing text posts from social media, which I'm making the assumption since how popular Reddit has become. It has become more than just a discussion forum.

Had an error downlaoding the VaderConstants Package and omitted it from the import

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#FIRST, we initialize VADER so we can use it within our Python script
sid = SentimentIntensityAnalyzer()

Manually change the sentiment attributed to certain words

In [ ]:
new_words = {
    
}
sid.lexicon.update(new_words)

In [ ]:
#Create a subset of the comments from the first 50 posts
df = comments
df.shape

In [ ]:
lemmatizer = nltk.WordNetLemmatizer()

In [ ]:
#The cleaning steps as outlined earlier
df.Reply = df.Reply.apply(lambda x: re.sub(r'[^\w\s]', '', x) )
df.Reply = df.Reply.apply(lambda x: ' '.join(nltk.word_tokenize(x)))
df.Reply = df.Reply.apply(lambda x: re.sub(r'\(?http\S+', '', x))
df.Reply = df.Reply.apply(lambda x: re.sub(r'[^\x00-\x7F]+',' ', x))
df.Reply = df.Reply.apply(lambda x: re.sub(r'^[ \t]+|[ \t]+$', '', x))
df.Reply = df.Reply.apply(lambda x: ' '.join(
    [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(x)]
        ) )


In [ ]:
df[['neg','neu','pos','compound']] = df.Reply.apply(lambda x: pd.Series(sid.polarity_scores(x)))

In [ ]:
df.head()

In [ ]:
#df.to_csv('df_comments.csv', index=False)